In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 28 01:32:54 2025

@author: mekie
"""


import datetime as dt
import dateutil as dtu

#renvoie lundi si nous somme Samedi ou dimanche
def adjust_date(unadj_date):
    # import datetime as dt
    import dateutil as dtu
    the_weekday = unadj_date.weekday()
    adj_date = unadj_date
    if the_weekday==5:
        adj_date = unadj_date + dtu.relativedelta.relativedelta(days=2)
    if the_weekday==6:
        adj_date = unadj_date + dtu.relativedelta.relativedelta(days=1)
    return adj_date


#renvoie vendredi si nous somme samedi ou dimanche (et lundi si nous sommes dimanche)
def adjust_date_bw(unadj_date):
    # import datetime as dt
    import dateutil as dtu
    the_weekday = unadj_date.weekday()
    adj_date = unadj_date
    if the_weekday==5:
        adj_date = unadj_date + dtu.relativedelta.relativedelta(days=-1)
    if the_weekday==6:
        adj_date = unadj_date + dtu.relativedelta.relativedelta(days=-2)
    return adj_date


#renvoie start_date + temps ou temps peut etre fonction de jours, mois ou année
def add_dmy(start_date, nb_days=0,  nb_months=0,  nb_years=0):
    delta_d = dtu.relativedelta.relativedelta(days=nb_days)
    delta_m = dtu.relativedelta.relativedelta(months=nb_months)
    delta_y = dtu.relativedelta.relativedelta(years=nb_years)
    end_date = start_date + delta_d + delta_m + delta_y
    return end_date

#renvoie spotdate + tenor
def tenor_to_date(spot_date, tenor):
    # import datetime as dt
    import dateutil as dtu
    
    # Days
    if tenor[-1]=='D':
        effective_date = spot_date + dtu.relativedelta.relativedelta(days=int(str.replace(tenor, 'D', '')))
    # Weeks
    if tenor[-1]=='W':
        effective_date = spot_date + dtu.relativedelta.relativedelta(days=7*int(str.replace(tenor, 'W', '')))
    # Months
    if tenor[-1]=='M':
        effective_date = spot_date + dtu.relativedelta.relativedelta(months=int(str.replace(tenor, 'M', '')))
    # Years
    if tenor[-1]=='Y':
        effective_date = spot_date + dtu.relativedelta.relativedelta(years=int(str.replace(tenor, 'Y', '')))
    return effective_date


def add_dates(the_DF, computation_date, spot_date, convention = "ACT/360"):
    the_DF['cmpt_date'] = computation_date
    the_DF['spot_date'] = spot_date
    the_DF['unadj_date'] = the_DF.apply(lambda x : tenor_to_date(x.spot_date, x.Tenor) , axis=1)
    the_DF['adj_date'] = the_DF.apply(lambda x : adjust_date(x.unadj_date) , axis=1)
    the_DF['day_diff'] = the_DF.apply(lambda x : (x.adj_date - x.spot_date).days , axis=1)
    the_DF['convention'] = the_DF.apply(lambda x : convention , axis=1)
    the_DF['alpha'] = the_DF.apply(lambda x : eval(str.replace(x.convention,'ACT','x.day_diff')) , axis=1)
    return the_DF

def get_30_360_daydiff(date1, date2):
    import dateutil as dtu
    nb_months = dtu.relativedelta.relativedelta(date2,date1).months
    daydiff_30_360 = (30 * nb_months) + (date2 - (date1 + dtu.relativedelta.relativedelta(months=nb_months) ) ).days
    return daydiff_30_360

